<a href="https://colab.research.google.com/github/DreamnVibes/RAG/blob/main/QA_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash

pip install haystack-ai
pip install "datasets>=2.6.1"
pip install "sentence-transformers>=4.1.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 530.1/530.1 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#Initialising the Document store that will contain data with its embeddings as indices
from haystack.document_stores.in_memory import InMemoryDocumentStore
document_store = InMemoryDocumentStore()

In [ ]:
#clear cache if needed
%%bash
rm -rf ~/.cache/huggingface/datasets

In [ ]:
#data fetch -- sample data from haystack (already preprocessed)
#wikipedia doc on 7 wonders of the world
from datasets import load_dataset
from haystack import Document
import pandas as pd

df = pd.read_parquet("hf://datasets/bilgeyucel/seven-wonders/data/train-00000-of-00001-4077bd623d55100a.parquet")
docs = [Document(content=row["content"], meta=row["meta"]) for _, row in df.iterrows()]



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
#import and warmup the embedding model
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
doc_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
doc_embedder.warm_up()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#Writing docs to document store (InMemoryDocumentStore)
docs_embed = doc_embedder.run(docs)
document_store.write_documents(docs_embed["documents"])

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

151

In [ ]:
#initialize retriever to retrieve data from document store
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
retriever = InMemoryEmbeddingRetriever(document_store)


In [ ]:
#define prompt template
from haystack.components.builders import ChatPromptBuilder
from haystack.dataclasses import ChatMessage

template = [
    ChatMessage.from_user(
        """
Given the following information, answer the question.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
"""
    )
]

prompt_builder = ChatPromptBuilder(template=template)

In [ ]:
!pip install -q google-generativeai

In [ ]:
#initialize an GoogleGenAIChatGenerator
from haystack.core.component import component
from haystack.dataclasses import ChatMessage

@component
class GeminiChatGenerator():
    def __init__(self, model_name="gemini-1.5-flash-latest", api_key=None):
        import google.generativeai as genai
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel(model_name)

    def warm_up(self):
        _ = self.model.generate_content("Hi")

    def run(self, messages: list[ChatMessage]):
        prompt = "\n".join([msg.text for msg in messages])
        response = self.model.generate_content(prompt)
        return {"replies": [ChatMessage.from_assistant(response.text)]}

In [ ]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

chat_generator = GeminiChatGenerator(api_key=GOOGLE_API_KEY)
chat_generator.warm_up()

In [ ]:
#Pipeline building
from haystack import Pipeline
from haystack.components.embedders import SentenceTransformersDocumentEmbedder, SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.builders import ChatPromptBuilder

basic_rag_pipeline = Pipeline()
# Adding components to pipeline
basic_rag_pipeline.add_component("text_embedder", SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2"))
basic_rag_pipeline.add_component("retriever", InMemoryEmbeddingRetriever(document_store=document_store))
basic_rag_pipeline.add_component("prompt_builder", ChatPromptBuilder(template=template))
basic_rag_pipeline.add_component("llm", GeminiChatGenerator(api_key=GOOGLE_API_KEY))

In [ ]:
# Connect the components
basic_rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
basic_rag_pipeline.connect("retriever.documents", "prompt_builder.documents")
basic_rag_pipeline.connect("prompt_builder", "llm")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: ChatPromptBuilder
  - llm: GeminiChatGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.messages (List[ChatMessage])

In [ ]:
# Ask question on document
query = "When was the great pyramid of giza built?"
result = basic_rag_pipeline.run(data={"text_embedder": {"text": query}, "prompt_builder": {"question": query}})
print(result["llm"]["replies"][0].text)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The text states the Great Pyramid of Giza was built in the early 26th century BC, over a period of around 27 years.  More precise dating is debated, with various authors providing estimates ranging from 2589 BC to 4235 BC.

